# Imports
All imports should be defined here to reduce clutter

In [5]:
import numpy as np
import pygame
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import matplotlib.pyplot as plot

from NovelSwarmBehavior.novel_swarms.config.EvolutionaryConfig import GeneticEvolutionConfig
from NovelSwarmBehavior.novel_swarms.config.WorldConfig import RectangularWorldConfig
from NovelSwarmBehavior.novel_swarms.config.defaults import ConfigurationDefaults
from NovelSwarmBehavior.novel_swarms.novelty.GeneRule import GeneRule
from NovelSwarmBehavior.novel_swarms.config.OutputTensorConfig import OutputTensorConfig
from generation.HaltedEvolution import HaltedEvolution

# Function Declarations

In [18]:
def initializeHaltedEvolution():
    agent_config = ConfigurationDefaults.DIFF_DRIVE_AGENT

    genotype = [
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
        GeneRule(_max=1.0, _min=-1.0, mutation_step=0.4, round_digits=4),
    ]

    phenotype = ConfigurationDefaults.BEHAVIOR_VECTOR

    world_config = RectangularWorldConfig(
        size=(500, 500),
        n_agents=30,
        behavior=phenotype,
        agentConfig=agent_config,
        padding=15
    )

    novelty_config = GeneticEvolutionConfig(
        gene_rules=genotype,
        phenotype_config=phenotype,
        n_generations=100,
        n_population=100,
        crossover_rate=0.7,
        mutation_rate=0.15,
        world_config=world_config,
        k_nn=15,
        simulation_lifespan=600,
        display_novelty=False,
        save_archive=False,
        show_gui=True
    )

    pygame.init()
    pygame.display.set_caption("Evolutionary Novelty Search")
    screen = pygame.display.set_mode((world_config.w, world_config.h))

    output_config = OutputTensorConfig(
        timeless=True,
        total_frames=80,
        steps_between_frames=2,
        screen=screen
    )

    halted_evolution = HaltedEvolution(
        world=world_config,
        evolution_config=novelty_config,
        output_config=output_config
    )

    return halted_evolution

In [42]:
class BehaviorIdentificationModel(torch.nn.Module):
    def __init__(self, n_classes=2):
        super(BehaviorIdentificationModel, self,).__init__()

        self.n_classes = n_classes

        self.conv1 = torch.nn.Conv2d(1, 1, 5, stride=2, padding=2)
        self.activation1 = torch.nn.ReLU()
        self.conv2 = torch.nn.Conv2d(1, 1, 3, stride=2, padding=1)
        self.activation2 = torch.nn.ReLU()
        self.conv3 = torch.nn.Conv2d(1, 1, 3, stride=2)
        self.activation3 = torch.nn.ReLU()

        self.pooling = torch.nn.MaxPool2d(3, stride=3)
        self.activation4 = torch.nn.ReLU()

        self.flatten = torch.nn.Flatten()

        self.linear1 = torch.nn.Linear(400, 200)
        self.activation5 = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(200, 100)
        self.activation6 = torch.nn.ReLU()
        self.classification_layer = torch.nn.Linear(100, self.n_classes)

        self.softmax = torch.nn.Softmax(dim=0)

    def forward(self, x):
        print(x.size())

        x = self.conv1(x)
        x = self.activation1(x)
        x = self.conv2(x)
        x = self.activation2(x)
        x = self.conv3(x)
        x = self.activation3(x)
        x = self.pooling(x)
        x = self.activation4(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.activation5(x)
        x = self.linear2(x)
        x = self.activation6(x)
        x = self.classification_layer(x)
        x = self.softmax(x)

        return x

    def increaseClassCount(self):
        self.n_classes += 1
        self.classification_layer = torch.nn.Linear(100, self.n_classes)

# Test Halted Evolution + TensorBoard
The following initializes a possible evolution, simulates the genome, and outputs the resulting frame to tensorboard

In [ ]:
def testTensorBoardAndEvolutionNexting():
    # Writer will output to ./runs/ directory by default
    writer = SummaryWriter()

    evolution = initializeHaltedEvolution()
    evolution.setup()

    frame, behavior_vector = evolution.next()
    frame = frame.astype(np.uint8)
    reshaped = np.reshape(frame, (1, 500, 500))

    # Tensorboard output
    writer.add_image('images', reshaped.astype(np.uint8), 0, dataformats="CWH")

    writer.close()
    pygame.quit()
testTensorBoardAndEvolutionNexting()

# Test Single Pass Through of the Network

In [ ]:
def testInputAndBackpropOfAlteredNetwork():
    evolution = initializeHaltedEvolution()
    evolution.setup()

    model = BehaviorIdentificationModel(n_classes=6)
    frame, _ = evolution.next()
    frame = frame.astype(np.uint8)
    reshaped = np.reshape(frame, (1, 500, 500))

    # Initialize a test loss function
    loss_fn = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

    tensor = torch.tensor(reshaped, dtype=torch.float32)
    y_hat1 = model.forward(tensor)
    print(y_hat1)

    y_truth = torch.tensor([[1, 0, 0, 0, 0, 0]], dtype=torch.float32)
    loss = loss_fn(y_hat1, y_truth)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    model.increaseClassCount()
    y_hat2 = model.forward(tensor)
    y_truth = torch.tensor([[1, 0, 0, 0, 0, 0, 0]], dtype=torch.float32)
    loss = loss_fn(y_hat2, y_truth)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(y_hat1, y_hat2)
    pygame.quit()
testInputAndBackpropOfAlteredNetwork()

Get Number of trainable parameters in the network

In [62]:
def printModelInformation():
    model = BehaviorIdentificationModel(n_classes=6)
    print(sum(p.numel() for p in model.parameters() if p.requires_grad))
    print(f"Model structure: {model}\n\n")
printModelInformation()

100952
Model structure: BehaviorIdentificationModel(
  (conv1): Conv2d(1, 1, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (activation1): ReLU()
  (conv2): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (activation2): ReLU()
  (conv3): Conv2d(1, 1, kernel_size=(3, 3), stride=(2, 2))
  (activation3): ReLU()
  (pooling): MaxPool2d(kernel_size=3, stride=3, padding=0, dilation=1, ceil_mode=False)
  (activation4): ReLU()
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear1): Linear(in_features=400, out_features=200, bias=True)
  (activation5): ReLU()
  (linear2): Linear(in_features=200, out_features=100, bias=True)
  (activation6): ReLU()
  (classification_layer): Linear(in_features=100, out_features=6, bias=True)
  (softmax): Softmax(dim=0)
)




In [63]:
def training_iteration(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def testing_iteration(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
